In [1]:
from pathlib import Path

import numba as nb
import numpy as np
import scipy.stats as stats

from natsort import natsorted

import matplotlib.pyplot as plt


#plt.style.use("mike")
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')


import astropy.units as u

import cmasher as cmr

green = "#33FF86"
purple = "#CE33FF"

%matplotlib widget
from cosmogrb.universe.survey import Survey

import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from zusammen.stan_models.stan_model import get_model
from zusammen import AnalysisBuilder, DataSet
from zusammen.spectral_plot import display_posterior_model_counts

from threeML import update_logging_level

import arviz as av


update_logging_level("FATAL")


from astromodels import Band_Calderone, PointSource, Model


from threeML import JointLikelihood, DataList, display_spectrum_model_counts

from astromodels import Cutoff_powerlaw

import popsynth as ps

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available


Import the survey and process the GRBs

In [ ]:
survey = Survey.from_file('data/survey.h5')
ab = AnalysisBuilder(survey, use_bb=True)

In [ ]:
ab.write_yaml("test_proc.yml")

In [2]:
ds = DataSet.from_yaml("test_proc.yml")

In [ ]:
ds.to_hdf5_file("sgrb.h5")

In [ ]:
ds.to_hdf5_file("sgrb.h5", sig_threshold=40)

In [ ]:
ds = DataSet.from_hdf5_file('sgrb.h5')

In [ ]:
ds.to_stan_dict()

Make Stan model

In [6]:
m = get_model("cpl_simple_chunked_gc")

In [7]:
m.clean_model()

In [8]:
m.build_model()

07:47:54 - cmdstanpy - INFO - compiling stan file /home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.stan to exe file /home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc
07:48:51 - cmdstanpy - INFO - compiled model executable: /home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc
07:48:51 - cmdstanpy - WARNING - Stan compiler has produced 26 warnings:
07:48:51 - cmdstanpy - WARNING - 
--- Translating Stan model to C++ code ---
bin/stanc --include-paths=/home/chrobin/Documents/zusammen/zusammen/stan_models --o=/home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.hpp /home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.stan
Warning in '/home/chrobin/Documents/zusammen/zusammen/stan_models/cpl.stan', line 35, column 41, included from
'/home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.stan', line 2, column 0: Declaration
    of arrays by placing bra

In [9]:
data = ds.to_stan_dict()

n_threads = 3
n_chains = 2

fit = m.model.sample(
    data=data,
    parallel_chains=n_chains,
    chains=n_chains,
    # inits= {'alpha':-1.},
    threads_per_chain=n_threads,
    seed=1234,
    iter_warmup=1000,
    iter_sampling=500,
    max_treedepth=12,
    show_progress=True,
)

07:48:52 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

11:41:51 - cmdstanpy - INFO - CmdStan done processing.


11:42:07 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: integrate: error estimate of integral 8.39823e-06 exceeds the given relative tolerance times norm of integral (in '/home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.stan', line 150, column 4 to column 105)
	Exception: integrate: error estimate of integral 6.15044e-06 exceeds the given relative tolerance times norm of integral (in '/home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.stan', line 150, column 4 to column 105)
	Exception: integrate: error estimate of integral 7.41529e-05 exceeds the given relative tolerance times norm of integral (in '/home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.stan', line 150, column 4 to column 105)
	Exception: integrate: error estimate of integral 3.51033e-06 exceeds the given relative tolerance times norm of integral (in '/home/chrobin/Documents/zusammen/zusammen/stan_models/cpl_simple_chunked_gc.

In [ ]:
m.model.sample?

Import Stan results into arviz

In [10]:
res = av.from_cmdstanpy(fit)

In [11]:
res.to_netcdf("test_inference_data_energy_flux_4.nc")

'test_inference_data_energy_flux_4.nc'

In [ ]:
res.sample_stats.treedepth.max()

In [ ]:
av.plot_trace(res)

In [ ]:
bc = Band_Calderone(opt=0, a =10, b=1e4)
bc.beta.fix = True

Load parameters  
    $\alpha$: parameter of CPL  
    $E_p = \log E_c$  
    $K$: normalization

In [ ]:
id = 0

In [ ]:
alpha = res.posterior.alpha.stack(sample=("chain", "draw")).values[id]

log_ec = res.posterior.log_ec.stack(sample=("chain", "draw")).values[id]

K  = res.posterior.K.stack(sample=("chain", "draw")).values[id]

epeak = res.posterior.epeak.stack(sample=("chain", "draw")).values[id]

energy_flux = res.posterior.energy_flux.stack(sample=("chain", "draw")).values[id]

gamma = res.posterior.gamma.stack(sample=("chain", "draw")).values[id]

log_Nrest = res.posterior.log_Nrest.stack(sample=("chain", "draw")).values[id]


div = res.sample_stats.diverging.stack(sample=("chain", "draw")).values

samples = np.vstack((K, alpha, 10.**log_ec))


dl = ds.get_data_list_of_interval(id)

In [ ]:
alpha

In [ ]:
div.sum()

In [ ]:
bc = Cutoff_powerlaw(piv=100)

bc.index.bounds = (None, None)
bc.K.bounds = (None, None)
bc.xc.bounds = (None, None)

model = Model(PointSource("ps",0,0, spectral_shape=bc))

In [ ]:
fig, ax = plt.subplots()


ax.scatter(ep, alpha, alpha=0.1)


In [ ]:
fig, ax = plt.subplots()


ax.scatter(K, alpha, alpha=0.1)

In [ ]:
display_posterior_model_counts(
    dl[1], model, samples.T[::20], min_rate=1e-99, shade=False
)

In [ ]:
cpl = Cutoff_powerlaw(piv=100,K=1e-1,xc=200)


dl = ds.get_data_list_of_interval(2)

model = Model(PointSource("ps",0,0, spectral_shape=cpl))

ba = JointLikelihood(model,DataList(*dl))

In [ ]:
ba.fit()

In [ ]:
display_spectrum_model_counts(ba)

In [ ]:
dl[0].view_count_spectrum()

In [ ]:
cpl